In [4]:
##Qmak_PrimaryCareFDM_FULL_V1.6.ipynb - last modified 202300629
## 20230302 - Added in srorganisation
## 20230208 - to account for thosepatients with no DOB etc as they are skeewing the data - they will be removed from the local persons table
## 20230209 - to account for persons not in obs period. Notes below 
## 20230615 - added DataDeleted clause to where queries. Thsi excludes items that have been superceded by new data.
## 20230629 - Amendded to reflect new Observation period calculation using new cb_fdm_managemet tables.
## 20230720 - Moved the section to update the dataset extract date into a new file.


#Builds the PrimaryCare FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't
# file location - https://github.com/ConnectedBradford/CB_FDM_PrimaryCare/tree/main

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) # required for datetime

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare"
targetdb <-'yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8'
targetdb <-gsub(' ','',targetdb)
print (sourcedb) 
print (targetdb)


eric <- paste("Set at  "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 


[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare"
[1] "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8"
[1] "Set at  2023-11-15 09:48:45.541252"


In [ ]:
#build care sites just for PrimaryCare 

sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
sql2<- paste('drop table if exists ' ,targetdb,'.tmp_orgs', sep = "")
sql3<- paste('drop table if exists ' ,targetdb,'.tmp_orgs2', sep = "")

#make the temp org tables 
sql4 <-paste('create table ' ,targetdb,'.tmp_orgs as select distinct IDOrganisation from ',sourcedb,'.tbl_SRCode', sep = "")
sql5 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationDoneAt from ',sourcedb,'.tbl_SRCode', sep = "")
#print (sql5)
sql6 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationRegisteredAt from ',sourcedb,'.tbl_SRCode', sep = "")
sql7 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationVisibleTo from ',sourcedb,'.tbl_SRCode', sep = "")
sql8 <-paste('create table ' ,targetdb,'.tmp_orgs2 as Select distinct IDOrganisation from ' ,targetdb,'.tmp_orgs', sep = "")
sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CB_FDM_MASTER.care_site a ,',targetdb,'.tmp_orgs2 b where a.care_site_source_value = b.IDOrganisation and a.place_of_service_source_value = \'GP practice site\'', sep = "")
#print (sql9)
sql10 <-paste('delete from ' ,targetdb,'.care_site where care_site_id = 236 or care_site_id = 886' , sep = "")

#Now Run them 
tb1 <- bq_project_query(project_id, sql1)
tb2 <- bq_project_query(project_id, sql2)
tb3 <- bq_project_query(project_id, sql3)
tb4 <- bq_project_query(project_id, sql4)
tb5 <- bq_project_query(project_id, sql5)
tb6 <- bq_project_query(project_id, sql6)
tb7 <- bq_project_query(project_id, sql7)
tb8 <- bq_project_query(project_id, sql8)
tb9 <- bq_project_query(project_id, sql9)
tb10 <- bq_project_query(project_id, sql10)

eric <- paste("care_site built "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#This query makes the observation period based on the persons table BUT adds in rules for this data extract.

#Qmak_PrimaryCareObservationPeriod_Part1

sql15 <-paste('drop table if exists ' ,targetdb,'.tmp_Eventdates' , sep = "")
sql16 <-paste('drop table if exists ' ,targetdb,'.tmp_EventdatesValid' , sep = "")
sql17 <-paste('truncate table ' ,targetdb,'.observation_period' , sep = "")

tb15 <-bq_project_query(project_id, sql15)
tb16 <-bq_project_query(project_id, sql16)
tb17 <-bq_project_query(project_id, sql17)

sql18 <-paste('create table ',targetdb,'.tmp_Eventdates as 
select a.person_id 
, a.DateEvent as EventDate
, "tbl_srcode" as source_table
from ',sourcedb,'.tbl_SRCode a  '  , sep = "")


sql19 <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, a.DateEvent as EventDate 
, "tbl_SRPrimaryCareMedication" as source_table
from ',sourcedb,'.tbl_SRPrimaryCareMedication a  '  , sep = "")

sql20 <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, a.DateEvent as EventDate 
, "tbl_SRReferralIn" as source_table
from ',sourcedb,'.tbl_SRReferralIn a '  , sep = "")


sql21 <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, a.DateEvent as EventDate 
, "tbl_SRReferralOut" as source_table
from ',sourcedb,'.tbl_SRReferralOut a '  , sep = "")

sql22 <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, a.DateBooked as EventDate 
, "tbl_SRVisit" as source_table
from ',sourcedb,'.tbl_SRVisit a'  , sep = "") 

sql23 <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, cast(a.DateStart as date) as EventDate 
, "tbl_SRAppointment" as source_table
from ',sourcedb,'.tbl_SRAppointment a ' , sep = "") 

sql23a <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, cast(a.DateEnd  as date)  as EventDate 
, "tbl_SRAppointment" as source_table
from ',sourcedb,'.tbl_SRAppointment a ' , sep = "") 

sql23b <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, cast(left(a.DateRegistration,10) as date)  as EventDate 
, "tbl_SRPatientRegistration" as source_table
from ',sourcedb,'.tbl_SRPatientRegistration a ' , sep = "") 

sql23b <-paste('insert into ',targetdb,'.tmp_Eventdates
select distinct a.person_id
, cast(left(a.DateDeRegistration,10) as date)  as EventDate 
, "tbl_SRPatientRegistration" as source_table
from ',sourcedb,'.tbl_SRPatientRegistration a ' , sep = "") 


tb18 <-bq_project_query(project_id, sql18)
tb19 <-bq_project_query(project_id, sql19)
tb20 <-bq_project_query(project_id, sql20)
tb21 <-bq_project_query(project_id, sql21)
tb22 <-bq_project_query(project_id, sql22)
tb23 <-bq_project_query(project_id, sql23)
tb23a <-bq_project_query(project_id, sql23a)
tb23b <-bq_project_query(project_id, sql23b)



eric <- paste("tmp_events tables built "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 

#Now build the persons table 

sql11 <-paste('drop table if exists ' ,targetdb,'.person', sep = "")
tb11 <- bq_project_query(project_id, sql11)


sql12 <-paste('create table ' ,targetdb,'.person 
as 
SELECT distinct a.person_id
, gender_concept_id
, year_of_birth
, month_of_birth
, day_of_birth
, birth_datetime
, death_datetime
, race_concept_id
, ethnicity_concept_id
, location_id
, provider_id
, care_site_id
, person_source_value
, gender_source_value
, gender_source_concept_id
, race_source_value
, race_source_concept_id
, ethnicity_source_value
, ethnicity_source_concept_id 
FROM yhcr-prd-phm-bia-core.CB_FDM_MASTER.person a
,',targetdb,'.tmp_Eventdates b #-- Amended tmp_EventDates as this gathers all the people  !
where a.person_id =  b.person_id' , sep = "")

tb12 <- bq_project_query(project_id, sql12)

eric <- paste("person table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric) 


sql24 <-paste('create table ',targetdb,'.tmp_EventdatesValid
as
select distinct a.person_id
,cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a ,' 
,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is not null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= date_add(e.death_datetime, INTERVAL 42 day) 
and a.EventDate <= (Select distinct updated_date from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list where dataset = \'CB_STAGING_DATABASE_PrimaryCare\')', sep = "")


#-- BUT this only includes persons with a death datetime
#-- So section below includes those with a null deathdatetime

sql25 <-paste('insert into ',targetdb,'.tmp_EventdatesValid
select distinct a.person_id
, cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a
,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= (Select distinct updated_date from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list where dataset = \'CB_STAGING_DATABASE_PrimaryCare\')', sep = "")

#third part pushes these into the observation_period table

sql26 <-paste('insert into ' ,targetdb,'.observation_period
select distinct
ROW_NUMBER() over (Order by person_id) as observation_period_id 
, person_id
, min(EventDate) as observation_period_start_date
, max(EventDate) as observation_period_end_date
,null as period_type_concept_id
from ',targetdb,'.tmp_EventdatesValid
group by person_id', sep = "")

#Finally Run these sql's


tb24 <-bq_project_query(project_id, sql24)
tb25 <-bq_project_query(project_id, sql25)
tb26 <-bq_project_query(project_id, sql26)

eric <- paste("observation period built " ,Sys.time() + hours(0) ,sep = "")
print(eric) 

# Now remove from person table where there is no observation period
# This happens where there is patient data but no actual records 
# or there is patient data but no dob 

sql26a <-paste(' delete from ' ,targetdb,'.person where person_id
not in (select person_id from ', targetdb, ' . observation_period)', sep = "")

tb26a <-bq_project_query(project_id, sql26a) 

eric <- paste("invalid people removed "  ,Sys.time() + hours(1) ,sep = "")
print(eric)


#Qapp_PrimaryCareFDM_SRAppointment
#Last Modified 20220831

sql27 <-paste('truncate table ' ,targetdb,'.tbl_srappointment', sep = "")

sql28 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DateAppointmentBooked  =  null where DateAppointmentBooked = \'NULL\'', sep = "")

sql29 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DatePatientArrival =  null where DatePatientArrival  =  \'NULL\'', sep = "")

sql30 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DatePatientSeen  = null where DatePatientSeen = \'NULL\'', sep = "")

sql31 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DateAppointmentCancelled = null where DateAppointmentCancelled = \'NULL\'', sep = "")

sql32 <-paste(' insert into ' ,targetdb,'.tbl_srappointment 
SELECT distinct sro.person_id  
, DateStart
, DateEnd
, sro.RowIdentifier  
, IDOrganisationVisibleTo
, DateStart
, DateEnd
, AppointmentStatus
, cast(DateAppointmentBooked as datetime) as DateAppointmentBooked
, cast(DatePatientArrival as datetime) as DatePatientArrival
, cast(DatePatientSeen as datetime) as DatePatientSeen
, FollowUpAppointment
, TelephoneAppointment
, IDClinician
, IDProfileClinician
, RotaName
, RotaType
, RotaLocation
, RotaCode
, IDRotaOwner
, IDProfileRotaOwner
, AllowsOverbooking
, BookingContactNumber
, cast(DateAppointmentCancelled as datetime) as DateAppointmentCancelled
, IDRota
, IDReferralIN
, IDPatient
, IDOrganisation
, IDOrganisationRegisteredAt
, cs3.care_site_id as care_site_id_VisibleTo
, cs1.care_site_id 
, cs2.care_site_id as care_site_RegisteredAt
, 0 as provider_id
#, cast(sro.provider_id as int64) as provider_id
FROM ', sourcedb,'.tbl_SRAppointment sro
, ' ,targetdb,'.observation_period obs
left join ' ,targetdb,'.care_site cs1
on sro.IDOrganisation = cs1.care_site_source_value
left join ' ,targetdb,'.care_site cs2
on sro.IDOrganisationRegisteredAt = cs2.care_site_source_value
left join ' ,targetdb,'.care_site cs3
on sro.IDOrganisationVisibleTo = cs3.care_site_source_value
where sro.person_id  = obs.person_id
and sro.DataDeleted is null
and cast(sro.DateEnd as date)  <= obs.observation_period_end_date
and cast(sro.DateStart as date)  >= obs.observation_period_start_date', sep = "")

#Now Run these 
tb27 <-bq_project_query(project_id, sql27)
tb28 <-bq_project_query(project_id, sql28)
tb29 <-bq_project_query(project_id, sql29)
tb30 <-bq_project_query(project_id, sql30)
tb31 <-bq_project_query(project_id, sql31)
tb32 <-bq_project_query(project_id, sql32)

eric <- paste("appointment table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRCode
#Last Modified 20220831
sql33 <-paste('truncate table ' ,targetdb,'.tbl_srcode', sep = "")

tb33 <-bq_project_query(project_id, sql33)

sql34 <-paste('insert into ' ,targetdb,'.tbl_srcode
(
  person_id
, tbl_srcode_start_date
, tbl_srcode_end_date
, RowIdentifier
, idorganisationvisibleto
, dateeventrecorded
, dateevent
, iddoneby
, idorganisationdoneat
, ctv3code
, ctv3text
, snomedcode
, numericvalue
, numericunit
, idorganisation
, idorganisationregisteredat
, idevent
, idreferralin
, idappointment
, idvisit
,care_site_id
,care_site_id_DoneAt
,care_site_id_RegisteredAt
,procedure_source_concept_id
,observation_source_concept_id
,condition_source_concept_id
,measurement_source_concept_id
)
SELECT distinct src.person_id 
, cast(DateEvent as datetime) as start_date
, cast(DateEvent as datetime) as end_date
, src.rowidentifier 
, IDOrganisationVisibleTo
, DateEventRecorded
, DateEvent
, cast(iddoneby as string) as IDDoneBy 
, idorganisationdoneat
, src.ctv3code
, ctv.CTV3Desc
, snm.SNOMEDCode
, cast(numericvalue as string) as numericvalue
, numericunit
, idorganisation
, idorganisationregisteredat
, cast(idevent as string) as idevent
, cast(idreferralin as string) as idreferralin
, cast(idappointment as string) as idappointment
, cast(idvisit as string) as idvisit
, cs1.care_site_id  as care_site_id
, cs3.care_site_id as care_site_DoneAt
, cs2.care_site_id as care_site_RegisteredAt
, con1.concept_id as procedure_source_concept_id
, con2.concept_id as observation_source_concept_id
, con3.concept_id as condition_source_concept_id
, con4.concept_id as measurement_source_concept_id
FROM ',sourcedb,'.tbl_SRCode src
, ' ,targetdb,'.observation_period obs
left join yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_CTV3Codes_Lookup ctv on src.ctv3code = ctv.CTV3Code
left join yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_CTV3ToSnomed_Map snm on src.ctv3code = snm.CTV3Code
left join ' ,targetdb,'.care_site cs1 on src.IDOrganisation = cs1.care_site_source_value
left join ' ,targetdb,'.care_site cs2 on src.IDOrganisationRegisteredAt = cs2.care_site_source_value
left join ' ,targetdb,'.care_site cs3 on src.IDOrganisationDoneAt = cs3.care_site_source_value
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con1 on snm.SNOMEDCode = con1.concept_code and ((con1.domain_id like \'Procedure%\' or con1.domain_id like \'Spec Anatomic%\') and con1.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr1 ON con1.concept_id = cr1.concept_id_1 and (cr1.relationship_id = \'Maps to\'and cr1.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con2 on snm.SNOMEDCode = con2.concept_code and (con2.domain_id like \'Observation%\' and con2.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr2 ON con2.concept_id = cr2.concept_id_1 and (cr2.relationship_id = \'Maps to\' and cr2.invalid_reason IS NULL )
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con3 on snm.SNOMEDCode = con3.concept_code  and ( con3.domain_id like \'Condition%\' and con3.vocabulary_id = \'SNOMED\') 
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr3 ON con3.concept_id = cr3.concept_id_1 and (cr3.relationship_id = \'Maps to\' and cr3.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con4 on snm.SNOMEDCode = con4.concept_code  and ( con4.domain_id like \'Measu%\' and con4.vocabulary_id = \'SNOMED\')
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr4 ON con4.concept_id = cr4.concept_id_1 and (cr4.relationship_id = \'Maps to\' and cr4.invalid_reason IS NULL)
where  src.person_id  = obs.person_id
#and src.DataDeleted is null removed as this is not indicating a null, its a duplicated potentially
and cast(DateEvent as date)   <= obs.observation_period_end_date
and cast(DateEvent as date)   >= obs.observation_period_start_date ', sep = "")

#Note: Snomed and ctv3code updated using left join on the Lookup tables in the lookup datasets

tb34 <-bq_project_query(project_id, sql34)
eric <- paste("srcode table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)
#Qapp_PrimaryCareFDM_SRImmunisation_V6
#Last Modified 20220831

sql35 <-paste('Update ',sourcedb,'.tbl_SRImmunisation
set dateexpiry = null where dateexpiry =\'NULL\'', sep = "")

sql36 <-paste('truncate table ' ,targetdb,'.tbl_srimmunisation', sep = "")

sql37 <-paste('insert into  ' ,targetdb,'.tbl_srimmunisation
SELECT distinct sri. person_id
,cast(dateevent as datetime) as start_date
,cast(dateevent as datetime) as end_date
, rowidentifier 
, idorganisationvisibleto
,  dateeventrecorded
,  dateevent
, idprofileenteredby
, iddoneby
, idorganisationdoneat
, idvaccination
, idimmunisationcontent
, dose
, location
, method
, cast(left(dateexpiry,19) as datetime) as dateexpiry
, immsreadcode
, vaccpart
, vaccbatchnumber
, vaccareacode
, vaccinationstatus
, idevent
, idpatient
, idorganisation
, idorganisationregisteredat 
, null as care_site_id
, null as care_site_Id_DoneAt
, null as care_site_id_RegisteredAt
FROM ',sourcedb,'.tbl_SRImmunisation sri
, ' ,targetdb,'.observation_period obs
where sri.person_id = obs.person_id
and sri.DataDeleted is null
and cast(DateEvent as date)  <= obs.observation_period_end_date
and cast(DateEvent as date)  >= obs.observation_period_start_date', sep = "")

tb35 <-bq_project_query(project_id, sql35)
tb36 <-bq_project_query(project_id, sql36)
tb37 <-bq_project_query(project_id, sql37)

eric <- paste("immunisation table built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

## Script is stopping here for some reason so will split

## Split here 

#now do immunisationcontent
sql38 <-paste('truncate table ' ,targetdb,'.tbl_srimmunisationcontent', sep = "")

sql39 <-paste('insert into ' ,targetdb,'.tbl_srimmunisationcontent
SELECT RowIdentifier,Name,Content,
DateRemoved
FROM ',sourcedb,'.tbl_SRImmunisationContent', sep = "")



tb38 <-bq_project_query(project_id, sql38)
tb39 <-bq_project_query(project_id, sql39)

eric <- paste("immunisation content built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Now do srorganisation


sql79<-paste('truncate table ' ,targetdb,'.tbl_srorganisation', sep = "")
tb79 <-bq_project_query(project_id, sql79)

sql80 <-paste('insert into ' ,targetdb,'.tbl_srorganisation
(rowidentifier,
name,
id,
housename,
housenumber,
nameofroad,
nameoflocality,
nameoftown,
nameofcounty,
fullpostcode,
telephone,
secondarytelephone,
fax,
madeobsolete,
idtrust,
idccg,
datadeleted)
select distinct sri.RowIdentifier, 
Name, 
ID, 
HouseName, 
HouseNumber, 
NameOfRoad, 
NameOfLocality, 
NameOfTown, 
NameOfCounty, 
FullPostCode, 
Telephone, 
SecondaryTelephone, 
Fax, 
MadeObsolete, 
cast(IDTrust as int64), 
cast(IDCcg as int64) , 
DataDeleted 
FROM ',sourcedb,'.tbl_SROrganisation sri
, ' ,targetdb,'.tmp_orgs2 cd
where sri.ID = cd.IDOrganisation
', sep = "")

tb80 <-bq_project_query(project_id, sql80)

eric <- paste("srorganisation built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRPatient_V6
#Last Modified 20220801

sql40 <-paste('truncate table ' ,targetdb,'.tbl_srpatient', sep = "")
tb40 <-bq_project_query(project_id, sql40)

sql41 <-paste('insert into ' ,targetdb,'.tbl_srpatient
(
 rowidentifier
,idorganisationvisibleto
,datebirth
,datedeath
,birthplace
,gender
,speaksenglish
,ssref
,spinematched
,ethnicity
,languagespoken
,soa
,ward
, person_id
#, care_site_id_visibleto
)
SELECT  RowIdentifier 
, IDOrganisationVisibleTo
, DateBirth
, DateDeath
, BirthPlace
, Gender
, SpeaksEnglish
, SSRef
, SpineMatched
, Ethnicity
, LanguageSpoken
, SOA
, Ward
, srp.person_id 
#, cs1.care_site_id as care_site_id_VisibleTo
FROM ',sourcedb,'.tbl_SRPatient srp
,  ' ,targetdb,'.person per 
where srp.person_id = per.person_id', sep = "")

tb41 <-bq_project_query(project_id, sql41)

eric <- paste("patient table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRPatientAddressHistory
#Last Modified 20220831

sql42 <-paste(' truncate table ' ,targetdb,'.tbl_srpatientaddresshistory', sep = "")

sql43a <-paste('Update ',sourcedb,'.tbl_SRPatientAddressHistory
set DateTo = null where DateTo =\'NULL\'', sep = "")

sql43 <-paste(' insert into ' ,targetdb,'.tbl_srpatientaddresshistory
SELECT  src.person_id
, cast(DateTo as DateTime) as fdm_start_date
, cast(DateTo as DateTime) as fdm_end_date
, RowIdentifier
, IDOrganisationVisibleTo
, DateEventRecorded
, null as DateEvent
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, PartialPostCode
, DateTo
, AddressType
, IDEvent
, IDPatient
, IDOrganisation
, CcgOfResidence
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_site_id_VisibleTo
 ,cs2.care_site_id  as care_site_id_DoneAt
 ,cs3.care_site_id  as care_site_id
 ,cs4.care_site_id  as  care_site_id_RegisteredAt
 ,null as provider_id
 #, cast(provider_id as int64) as prpovid
 FROM ',sourcedb,'.tbl_SRPatientAddressHistory src
 , ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on src.IDOrganisationVisibleTo = cs1.care_site_source_value
 left join ' ,targetdb,'.care_site cs2
 on src.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on src.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on src.IDOrganisationRegisteredAt = cs4.care_site_source_value
where src.person_id = obs.person_id
and cast(left(DateTo,10) as date)<= obs.observation_period_end_date
and cast(left(DateTo,10) as date)  >= obs.observation_period_start_date', sep = "")

tb42 <-bq_project_query(project_id, sql42)
tb43a <-bq_project_query(project_id, sql43a)
tb43 <-bq_project_query(project_id, sql43)


eric <- paste("PrimaryCareFDM_SRPatientAddressHistory done "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRPatientRegistration
#Last modified 20220318 - JDB

sql44 <-paste('Update ',sourcedb,'.tbl_SRPatientRegistration
set DateRegistration = null where DateRegistration = \'NULL\'', sep = "")
tb44 <-bq_project_query(project_id, sql44)

sql45 <-paste('Update ',sourcedb,'.tbl_SRPatientRegistration
set DateDeRegistration = null where DateDeRegistration = \'NULL\'', sep = "")
tb45 <-bq_project_query(project_id, sql45)

sql46 <-paste(' truncate table ' ,targetdb,'.tbl_srpatientregistration', sep = "")
tb46 <-bq_project_query(project_id, sql46)

sql47 <-paste(' insert into ' ,targetdb,'.tbl_srpatientregistration
SELECT distinct cast(src.person_id as int64) as person_id 
, cast(left(DateRegistration,10) as date) as fdm_datestart
, cast(left(DateRegistration,10) as date) as fdm_dateend
, RowIdentifier
, IDOrganisationVisibleTo
, dateeventrecorded
, IDProfileEnteredBy
, cast(left(DateRegistration,10) as date)  as DateRegistration
, cast(left(DateDeRegistration,10) as date) as DateDeRegistration
, RegistrationStatus
, PreferredPharmacy
, cast(IDPatient as int64) as IDPatient
, IDOrganisation
, IDOrganisationRegisteredAt
, cs1.care_site_id  as care_site_id_VisibleTo   
, cs3.care_site_id  as care_site_id_DoneAt
, cs4.care_site_id  as care_site_id_RegisteredAt
FROM ',sourcedb,'.tbl_SRPatientRegistration src
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on src.IDOrganisationVisibleTo = cs1.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on src.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on src.IDOrganisationRegisteredAt = cs4.care_site_source_value
where src.person_id = obs.person_id
and cast(left(DateRegistration,10) as date)  <= obs.observation_period_end_date 
and cast(left(DateRegistration,10) as date)  >= obs.observation_period_start_date', sep = "")


tb47 <-bq_project_query(project_id, sql47)
eric <- paste("PrimaryCareFDM_SRPatientRegistration "  ,Sys.time() + hours(0) ,sep = "")
print(eric)


#Qapp_PrimaryCareFDM_SRPrimaryCareMedication
#Last Modified 20220831 - JDB 

sql48 <-paste(' truncate table ' ,targetdb,'.tbl_srprimarycaremedication', sep = "")

sql49 <-paste(' insert into ' ,targetdb,'.tbl_srprimarycaremedication
(
person_id
, tbl_srprimarycaremedication_start_date
, tbl_srprimarycaremedication_end_date
, rowidentifier 
, idorganisationvisibleto
, dateeventrecorded
, dateevent
, iddoneby
, idorganisationdoneat
, idmultilexproduct
, idmultilexpack
, idmultilexdmd
, nameofmedication
, datemedicationstart
, datemedicationend
, medicationdosage
, medicationquantity
, issupervised
, superviseddose
, unsuperviseddose
, isrepeatmedication
, isothermedication
, isdentalmedication
, ishospitalmedication
, idorganisationregisteredat 
, care_site_id_VisibleTo   
, care_site_id_DoneAt
, care_site_id_RegisteredAt
,drug_source_concept_id
,device_source_concept_id
)
SELECT distinct srm.person_id
, cast(dateevent as datetime)  as fdm_start_date
, cast(dateevent as datetime)  as fdm_end_date
, cast(rowidentifier as string) as rowidentifier 
, idorganisationvisibleto
, dateeventrecorded
, dateevent
, iddoneby
, idorganisationdoneat
, idmultilexproduct
, idmultilexpack
, idmultilexdmd
, nameofmedication
, datemedicationstart
, datemedicationend
, medicationdosage
, medicationquantity
, issupervised
, superviseddose
, unsuperviseddose
, isrepeatmedication
, isothermedication
, isdentalmedication
, ishospitalmedication
, idorganisationregisteredat 
, cs1.care_site_id  as care_site_id_VisibleTo   
, cs2.care_site_id  as care_site_id_DoneAt
, cs4.care_site_id  as care_site_id_RegisteredAt
,con1.concept_id as drug_source_concept_id
,con2.concept_id as device_source_concept_id
FROM ',sourcedb,'.tbl_SRPrimaryCareMedication srm 
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on srm.IDOrganisationVisibleTo = cs1.care_site_source_value
  left join ' ,targetdb,'.care_site cs2
 on srm.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on srm.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on srm.IDOrganisationRegisteredAt = cs4.care_site_source_value
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con1
on srm.IDMultiLexDMD = con1.concept_code and (con1.domain_id like \'Drug%\' and con1.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr1
ON con1.concept_id = cr1.concept_id_1 and (cr1.relationship_id = \'Maps to\'and cr1.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept con2
on srm.IDMultiLexDMD = con2.concept_code and (con2.domain_id like \'Device%\' and con2.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept_relationship cr2
ON con2.concept_id = cr2.concept_id_1 and (cr2.relationship_id = \'Maps to\'and cr2.invalid_reason IS NULL)
where srm.person_id  = obs.person_id
and cast(DateEvent as date)  <= obs.observation_period_end_date
and cast(DateEvent as date)   >= obs.observation_period_start_date', sep = "")

tb48 <-bq_project_query(project_id, sql48)
tb49 <-bq_project_query(project_id, sql49)

eric <- paste("primary care meds table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRReferralIn
#Last Modified 20220831

sql50 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateReferral = null where DateReferral = \'NULL\'', sep = "")
sql51 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateOnReferralLetter = null where DateOnReferralLetter = \'NULL\'', sep = "")
sql52 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateDischarge = null where DateDischarge = \'NULL\'', sep = "")
sql53 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateOfDecision = null where DateOfDecision = \'NULL\'', sep = "")

sql54 <-paste(' truncate table ' ,targetdb,'.tbl_srreferralin ', sep = "")

sql55 <-paste(' insert into ' ,targetdb,'.tbl_srreferralin 
(
 person_id
,tbl_srreferralin_start_date
,tbl_srreferralin_end_date
,rowidentifier
,idorganisationvisibleto
,dateeventrecorded
,dateevent
,idprofileenteredby
,iddoneby
,textualeventdoneby
,idorganisationdoneat
,category
,primaryreason
,datereferral
,dateonreferralletter
,source
,serviceoffered
,servicetype
,rereferral
,urgency
,idcodeprimarydiagnosis
,referralcode
,idreferrallocal
,reasonforservicedelay
,outcome
,defaultcontactlocation
,datedischargescheduled
,datedischarge
,dischargereason
,dischargelocation
,idreferrer
,sourcetablereferrer
,patientaware
,dateofaction
,activestatus
,referredtospeciality
,referredtotreatmentcode
,idreferredto
,sourcetablereferrerto
,referralpriority
,referralmedium
,dateofdecision
,daycarereferral
,maternityreferral
,commissioner
,clientcategory
,primarysupportreason
,idcaseload
,idevent
,idpatient
,idorganisation 
)
SELECT sri.person_id
, cast(dateevent as datetime) as fdm_start_date
, cast(dateevent as datetime) as fdm_end_date
, cast(RowIdentifier as string)
, IDOrganisationVisibleTo
, cast(DateEventRecorded as Date) 
, cast(DateEvent as Date) 
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, Category
, PrimaryReason
, cast(DateReferral as date) as DateReferral
, cast(DateOnReferralLetter as date) as DateOnReferralLetter
, Source
, ServiceOffered
, ServiceType
, ReReferral
, Urgency
, IDCodePrimaryDiagnosis
, ReferralCode
, IDReferralLocal
, ReasonForServiceDelay
, Outcome
, DefaultContactLocation
, DateDischargeScheduled
, cast(DateDischarge as date) as DateDischarge
, DischargeReason
, DischargeLocation
, IDReferrer
, SourceTableReferrer
, PatientAware
, DateOfAction
, ActiveStatus
, ReferredToSpeciality
, ReferredToTreatmentCode
, IDReferredTo
, SourceTableReferrerTo
, ReferralPriority
, ReferralMedium
, cast(DateOfDecision as date) as DateOfDecision
, DayCareReferral
, MaternityReferral
, Commissioner
, ClientCategory
, PrimarySupportReason
, IDCaseload
, IDEvent
, IDPatient
, IDOrganisation 
FROM ',sourcedb,'.tbl_SRReferralIn sri
, ' ,targetdb,'.observation_period obs
where  sri.person_id  = obs.person_id
and cast(DateEvent as date)  <= obs.observation_period_end_date
and cast(DateEvent as date)   >= obs.observation_period_start_date', sep = "")

eric <- paste("referralin table built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)
#Qapp_PrimaryCareFDM_SRReferralOUT
#Last Modified 2022318

sql56 <-paste(' truncate table  ' ,targetdb,'.tbl_srreferralout', sep = "")

sql57 <-paste(' insert into  ' ,targetdb,'.tbl_srreferralout
(
    person_id
, tbl_srreferralout_start_date
, tbl_srreferralout_end_date
, rowidentifier
, idorganisationvisibleto
, dateeventrecorded
, dateevent
, idprofileenteredby
, iddoneby
, textualeventdoneby
, idorganisationdoneat
, typeofreferral
, reason
, idprofilereferrer
, serviceoffered
, rereferral
, urgency
, primarydiagnosis
, recipientid
, recipientidtype
, idevent
, idpatient
, idorganisation
, m101140reasonforoutofareareferraladultacutementalhealth
, idorganisationregisteredat
, care_site_id_VisibleTo
, care_site_id_DoneAt
, care_site_id
, care_site_id_RegisteredAt
, provider_id
)
SELECT  sro.person_id
, cast(DateEvent as Date) as fdm_start_date
, cast(DateEvent as Date) as fdm_end_date
, cast(RowIdentifier as int64) as rowidentifier
, IDOrganisationVisibleTo
, cast(DateEventRecorded as Date)
, cast(DateEvent as Date) 
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, TypeOfReferral
, Reason
, IDProfileReferrer
, ServiceOffered
, ReReferral
, Urgency
, PrimaryDiagnosis
, RecipientID
, RecipientIDType
, IDEvent
, IDPatient
, IDOrganisation
, M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_care_site_id_VisibleTo
, cs2.care_site_id as care_site_id_DoneAt
, cs3.care_site_id as care_site_id
, cs4.care_site_id as  care_site_id_RegisteredAt
, null as provider_id
FROM ',sourcedb,'.tbl_SRReferralOut sro
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on sro.IDOrganisationVisibleTo = cs1.care_site_source_value
  left join ' ,targetdb,'.care_site cs2
 on sro.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on sro.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on sro.IDOrganisationRegisteredAt = cs4.care_site_source_value
where  sro.person_id  = obs.person_id
and  cast(dateevent as date) <= obs.observation_period_end_date
and  cast(dateevent as date) >= obs.observation_period_start_date', sep = "")


tb50 <-bq_project_query(project_id, sql50)
tb51 <-bq_project_query(project_id, sql51)
tb52 <-bq_project_query(project_id, sql52)
tb53 <-bq_project_query(project_id, sql53)
tb54 <-bq_project_query(project_id, sql54)
tb55 <-bq_project_query(project_id, sql55)
tb56 <-bq_project_query(project_id, sql56)
tb57 <-bq_project_query(project_id, sql57)

eric <- paste("referralout table built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#Now Append Lookuop tables - prefix with cb


sql98 <-paste(' drop  table if exists ',targetdb,'.cb_srmedicationreadcodedetails ', sep = "")

sql99 <-paste(' create table ',targetdb,'.cb_srmedicationreadcodedetails as
SELECT distinct 
RowIdentifier as rowidentifier, 
IDMultiLexProduct as idmultilexdmdproduct, 
DrugReadCode as drugreadcode, 
DrugReadCodeDesc as drugreadcodedesc
FROM ',sourcedb,'.tbl_SRMedicationReadCodeDetails', sep = "")

tb98 <-bq_project_query(project_id, sql98)
tb99 <-bq_project_query(project_id, sql99)

eric <- paste("srmedicationread code details built "  ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_SRVisit
#Last Modified 20220831 - JDB
sql58 <-paste(' truncate table ' ,targetdb,'.tbl_srvisit', sep = "")

sql59 <-paste(' insert into ' ,targetdb,'.tbl_srvisit 
(
    person_id
,tbl_srvisit_start_date
,tbl_srvisit_end_date
,rowidentifier
,idorganisationvisibleto
,dateeventrecorded
,idprofileenteredby
,datebooked
,daterequested
,currentstatus
,idprofilerequested
,idprofileassigned
,followupdetails
,idreferralin
,idpatient
,idorganisation
,duration
,idorganisationregisteredat
, care_site_id_VisibleTo
, care_site_id
, care_site_id_RegisteredAt
#, provider_id
)
SELECT distinct srv.person_id
, cast(DateBooked as date) as fdm_start_date
, cast(DateBooked as date) as fdm_end_date
, srv.RowIdentifier
, IDOrganisationVisibleTo
, DateEventRecorded
, IDProfileEnteredBy
, DateBooked
, DateRequested
, CurrentStatus
, IDProfileRequested
, IDProfileAssigned
, FollowUpDetails
, IDReferralIN
, IDPatient
, IDOrganisation
, Duration
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_site_id_VisibleTo
, cs3.care_site_id as care_site_id
, cs4.care_site_id as care_site_id_RegisteredAt
#,  provider_id
FROM ',sourcedb,'.tbl_SRVisit srv
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on srv.IDOrganisationVisibleTo = cs1.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on srv.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on srv.IDOrganisationRegisteredAt = cs4.care_site_source_value
where  srv.person_id  = obs.person_id
and cast(srv.DateBooked as date) <= obs.observation_period_end_date
and cast(srv.DateBooked as date) >= obs.observation_period_start_date', sep = "")

tb58 <-bq_project_query(project_id, sql58)
tb59 <-bq_project_query(project_id, sql59)

eric <- paste("visit table built " ,Sys.time() + hours(0) ,sep = "")
print(eric)

#Qapp_PrimaryCareFDM_StaffMemberProfileRole
#Last Modified 20220131 - JDB

sql60 <-paste(' truncate table  ' ,targetdb,'.tbl_srstaffmemberprofile', sep = "")

sql61 <-paste(' insert into ' ,targetdb,'.tbl_srstaffmemberprofile
SELECT  RowIdentifier, 
DateProfileCreated, 
IDProfileCreatedBy, 
IDStaffMemberProfileRole, 
StaffRole, DateEmploymentStart, 
DateEmploymentEnd, 
PPAID, 
GPLocalCode, 
IDStaffMember, 
IDOrganisation, 
GmpID,
cs1.care_site_id as care_site_id
FROM ',sourcedb,'.tbl_SRStaffMemberProfile smpr
left join ' ,targetdb,'.care_site cs1
 on smpr.IDOrganisation = cs1.care_site_source_value
where smpr.DataDeleted is null', sep = "")

tb60 <-bq_project_query(project_id, sql60)
tb61 <-bq_project_query(project_id, sql61)
eric <- paste("staffmemberprofile table built" ,Sys.time() + hours(0) ,sep = "")
print(eric)

eric <- paste("starting visit builders " ,Sys.time() + hours(0) ,sep = "")
print(eric)

[1] "care_site built 2023-11-15 09:49:17.870621"
[1] "tmp_events tables built 2023-11-15 09:50:04.816398"
[1] "person table built 2023-11-15 09:50:17.115561"
[1] "observation period built 2023-11-15 09:50:40.457963"


In [7]:
#VISITBuilders
#These are added value scripts that build pseudo visit occurences and visit detail tables. 
#Finally – start the Visit Builder scripts 
#Qapp_PrimaryCareFDM_VisitBuilder_Part1
#Last modified 20220831 - JDB

sql62 <-paste(' truncate table ' ,targetdb,'.tbl_visit_builder', sep = "")
tb62 <-bq_project_query(project_id, sql62)

sql63 <-paste(' insert into ' ,targetdb,'.tbl_visit_builder
(
  rowidentifier
, idvisit
, idappointment
)
select distinct  a.rowidentifier 
      ,cast(a.idvisit as int64) 
      ,cast(a.idappointment as int64) 
    from ' ,targetdb,'.tbl_srcode a
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where a.person_id = c.person_id
  and c.person_id = obs.person_id
  and a.dateevent <= obs.observation_period_end_date
  and a.dateevent >= obs.observation_period_start_date', sep = "")

tb63 <-bq_project_query(project_id, sql63)

sql64 <-paste('Update ' ,targetdb,'.tbl_visit_builder 
set visit_type = null 
where visit_type is not null', sep = "") 

tb64 <-bq_project_query(project_id, sql64)

#print (sql64) 

sql65 <-paste('Update ' ,targetdb,'.tbl_visit_builder
set visit_type= \'detail\'
where idvisit > 0
or idappointment > 0', sep = "")
tb65 <-bq_project_query(project_id, sql65)


sql66 <-paste('Update ' ,targetdb,'.tbl_visit_builder
set visit_type= \'occ\'
where visit_type is null ', sep = "")

tb66 <-bq_project_query(project_id, sql66)

#Qapp_PrimaryCareFDM_Visit_Detail_srcVisits
#Last modified 20220831 - JDB

sql67 <-paste(' truncate table ' ,targetdb,'.visit_detail', sep = "") 

sql68 <-paste(' insert into ' ,targetdb,'.visit_detail 
(      visit_detail_id 
      ,person_id
      ,rowid      
      ,table
      ,clinicaldataid 
      ,visitstartdate
      ,visitstart
      ,visitenddate
      ,visitend
      ,visitduration
      ,visitstartplace
      ,visitendplace
      ,visitlocation
      ,visitprovider
    ,visitdesc
    ,visit_detail_concept_id
    ,care_site_id
)
select distinct ROW_NUMBER() over (Order by c.rowidentifier) as visit_detail_id
  , b.person_id 
  , cast(c.rowidentifier as string) as rowid  
  , \'primary care srvisit\' as table
  , cast(c.IDVisit as string) as clinicaldataid
  ,cast(a.Daterequested as date) as visitstartdate
  ,a.Daterequested as visitstartdatetime
  ,cast(a.Daterequested as date)  as visitenddate
  ,a.Daterequested as visitenddatetime
  ,a.Duration
  ,a.IDOrganisation as startplace
  ,a.IDOrganisation as endplace
  ,a.IDOrganisation as visitlocation
  ,d.StaffRole as visitprovider
  ,\'primary care srvisit detail\' as visitdesc
  ,38004693 as visit_detail_concept_id
  ,cs1.care_site_id  as care_site_id
   from ' ,targetdb,'.tbl_srvisit  a 
  inner join ' ,targetdb,'.tbl_srpatient  b
  on a.person_id = b.person_id  
  inner join ' ,targetdb,'.tbl_visit_builder c
  on a.rowidentifier = c.IDVisit
  left join ' ,targetdb,'.tbl_srstaffmemberprofile d
  on (a.IDProfileAssigned = d.IDStaffMemberProfileRole and a.IDOrganisation = d.IDOrganisation)
   left join ' ,targetdb,'.care_site cs1
 on a.IDOrganisation = cs1.care_site_source_value
  where c.visit_type = \'detail\'
  and cast(c.IDVisit as string)  not in 
(Select RowID from ' ,targetdb,'.visit_detail 
where Table = \'primary care srvisit\')', sep = "")

tb67 <-bq_project_query(project_id, sql67)
tb68 <-bq_project_query(project_id, sql68)


#Appends the appointments to the visit detailtable 

# Qapp_PrimaryCareFDM_Visit_Detail_srcAppointments

#Last modified 20220831 - JDB

sql69 <-paste('delete from ' ,targetdb,'.visit_detail 
where VisitDesc !=\'primary care srvisit detail\'', sep = "")

sql70 <-paste(' insert into ' ,targetdb,'.visit_detail 
 (     visit_detail_id 
      ,person_id
      ,rowid
      ,table
      ,clinicaldataid 
      ,visitstartdate
      ,visitstart
      ,visitenddate
      ,visitend
      ,visitduration
      ,visitstartplace
      ,visitendplace
      ,visitlocation
      ,visitprovider
      ,visitdesc
      ,visit_detail_concept_id
    ,care_site_id
)
Select distinct ROW_NUMBER() over (Order by c.rowidentifier) as visit_detail_id
      , a. person_id
      , cast(c.RowIdentifier as string)  as rowid 
  , \'primary care srappointment\' as sourcetable
  ,cast(c.IDAppointment as string)   as clinicaldataid 
     ,cast(a.datepatientarrival as date)
      ,cast(a.datepatientarrival as datetime)  as startdatetime
      ,cast(a.datepatientseen as date)
      ,cast(a.datepatientseen as datetime) as enddatetime
     ,cast(TIMESTAMP_DIFF(cast(a.DatePatientSeen as datetime) ,cast(a.DatePatientArrival as datetime) , MINUTE) as string) as duration
      ,a.IDOrganisation as visitstartplace
      ,a.IDOrganisation as visitendplace
      ,a.IDOrganisation as visitlocation 
      ,StaffRole as visitprovider
      ,case when a.TelephoneAppointment = \'true\' then \'telehealth\' 
      when a.RotaName like \'%home%visit%\' then \'home visit\'
      when a.RotaType like \'%home%visit%\' then \'home visit\' 
      when a.RotaName like \'%session%\' then \'clinic or group practice\'
      when a.RotaType like \'%session%\' then \'clinic or group practice\' 
      when a.RotaName like \'%clinic%\' then \'clinic or group practice\'
      when a.RotaType like \'%clinic%\' then \'clinic or group practice\' 
      when a.RotaName like \'%surgery%\' then \'clinic or group practice\'
      when a.RotaType like \'%surgery%\' then \'clinic or group practice\' 
      else a.RotaName end as visitdesc
      ,38004693 as visit_detail_concept_id
    ,cs1.care_site_id as care_site_id
        from ' ,targetdb,'.tbl_srappointment a
        inner join ' ,targetdb,'.tbl_srpatient b
        on a.person_id = b.person_id            
        inner join ' ,targetdb,'.tbl_visit_builder c
        on a.RowIdentifier = c.IDAppointment
        left join ' ,targetdb,'.tbl_srstaffmemberprofile d
        on cast(a.IDProfileClinician as int64)  = d.RowIdentifier
           left join ' ,targetdb,'.care_site cs1
 on a.IDOrganisation = cs1.care_site_source_value
        where a.DateAppointmentCancelled is null
       and a.DatePatientseen is not null ', sep = "")

tb69 <-bq_project_query(project_id, sql69)
tb70 <-bq_project_query(project_id, sql70)


eric <- paste("visit detail built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)
#Qapp_PrimaryCareFDM_Visit_Occurrences_ALL
#Last modified - 20220831 - JDB

#amended to sql <-paste(' insert everything into the occurrences table 

sql71 <-paste(' truncate table ' ,targetdb,'.visit_occurrence ', sep = "")

sql72 <-paste(' insert into ' ,targetdb,'.visit_occurrence 
(      person_id
      ,visit_occurrence_id
      ,rowid      -- the new visit table rowidentifier
      ,table#
      ,clinicaldataid -- the actual rowidentifier from the srcode table
      ,visitstartdate
      ,visitstartdatetime
      ,visitenddate
      ,visitenddatetime
      ,visitduration
      ,visitstartplace
      ,visitendplace
      ,visitlocation
      ,care_site_id
      ,visitprovider
#       --,provider_id
      ,visitdesc
      ,visit_source_concept_id
)
select distinct a.person_id
      ,c.RowIdentifier   as visit_occurrence_id
      ,cast(a.RowIdentifier as string)  as rowid 
        , \'primary care srcode\' as table
        ,cast(a.RowIdentifier as string)  as clinicaldataid 
      ,cast(a.DateEvent as Date) as visitstartdate
        ,cast(a.DateEvent as DateTime) as visitstartdatetime
      ,cast(a.DateEvent as Date) as visitenddate
        ,cast(a.DateEvent as DateTime) as visitenddatetime
      ,\'0\' as duration
        ,a.IDOrganisationDoneAt as visitstartplace
        ,a.IDOrganisationDoneAt as visitendplace
        ,a.IDOrganisationDoneAt as visitlocation
        ,d.care_site_id  as care_site_id
        ,a.IDDoneBy as visitprovider
       # --,e.provider_id as provider_id
        ,a.ctv3text as visitdesc
        ,38004693 as visit_source_concept_id
   from ' ,targetdb,'.tbl_srcode  a 
  inner join ' ,targetdb,'.person b
  on a.person_id = b.person_id 
  inner join ' ,targetdb,'.tbl_visit_builder c
  on a.RowIdentifier = c.RowIdentifier
  left join ' ,targetdb,'.care_site d
  on a.IDOrganisationDoneAt = d.care_site_source_value', sep = "")


tb71 <-bq_project_query(project_id, sql71)
tb72 <-bq_project_query(project_id, sql72)

eric <- paste("visit occurrence built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)
#This section builds the imosphere version of the visit_occurrence table

sql73 <-paste('drop table if exists ',targetdb,'.visit_occurrence_imo', sep = "")

sql74 <-paste('create table ',targetdb,'.visit_occurrence_imo
(
  visit_occurrence_id           INT64     not null ,
  person_id                     INT64     not null ,
  visit_concept_id              INT64     not null ,
  visit_start_DATE              DATE      not null ,
  visit_start_DATETIME          DATETIME           ,
  visit_end_DATE                DATE      not null ,
  visit_end_DATETIME            DATETIME           ,
  visit_type_concept_id         INT64     not null ,
  provider_id                   INT64              ,
  care_site_id                  STRING             ,
  visit_source_value            STRING             ,
  visit_source_concept_id       INT64              ,
  admitting_source_concept_id   INT64              ,
  admitting_source_value        STRING             ,
  discharge_to_concept_id       INT64              ,
  discharge_to_source_value     STRING             ,
  preceding_visit_occurrence_id INT64
)', sep = "")

tb73 <-bq_project_query(project_id, sql73)
tb74 <-bq_project_query(project_id, sql74)




sql75 <-paste(' insert into ',targetdb,'.visit_occurrence_imo
(
  visit_occurrence_id,
person_id,
visit_concept_id,
visit_start_date,
visit_start_datetime,
visit_end_date,
visit_type_concept_id,
care_site_id
)
select RowIdentifier
,vis.person_id
,9202 as visit_concept_id
,cast(DateBooked as date)  
,DateBooked
,cast(DateBooked as Date) 
,32817 as visit_type_concept_id
,IDOrganisation
FROM ',sourcedb,'.tbl_SRVisit vis
 , ' ,targetdb,'.observation_period obs
  where vis.person_id = obs.person_id
  and vis.DateBooked <= obs.observation_period_end_date
  and vis.DateBooked >= obs.observation_period_start_date', sep = "")
tb75 <-bq_project_query(project_id, sql75)

#New section for configured ListOption
eric <- paste("visit occurrence imosphere version built"  ,Sys.time() + hours(1) ,sep = "")
print(eric)

sql76 <-paste('drop table if exists ' ,targetdb,'.tbl_srconfiguredlistoption', sep = "")
tb76 <-bq_project_query(project_id, sql76)

sql77 <-paste('create table ' ,targetdb,'.tbl_srconfiguredlistoption as select cast(RowIdentifier as int64) as RowIdentifier
      ,cast(ConfiguredList as int64) as ConfiguredList
      ,ConfiguredListOption
      ,CDSCode
      ,CAMHSCode
      ,MHSDSCode from ',sourcedb,'.tbl_SRConfiguredListOption', sep = "")
tb77 <-bq_project_query(project_id, sql77)


eric <- paste("FDM Build of ", targetdb ," finished" ,Sys.time() + hours(0) ,sep = "")
print (eric)

[1] "visit detail built 2023-10-19 16:09:26.307683"
[1] "visit occurrence built 2023-10-19 16:09:53.184937"
[1] "visit occurrence imosphere version built2023-10-19 16:09:59.297552"
[1] "FDM Build of yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8 finished2023-10-19 16:10:03.834765"
